In [1]:
!pip install wordcloud
!pip install tensorflow
!pip install keras
!pip install emoji
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import re
import string
import emoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.impute import SimpleImputer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from wordcloud import WordCloud
import matplotlib.pyplot as plt
nltk.download('wordnet')
nltk.download('punkt')

2024-09-10 13:58:06.618514: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package wordnet to /Users/firman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/firman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Making data frame to predict the sentiment of the reviews manually
data = {'Review': ['I was at the starbucks today and the coffee was amazing',
                    'The baristas are rude']}


df = pd.DataFrame(data)


In [3]:
# Making cleaning funcion
def cleaning (text):
    text = re.sub('[^a-zA-Z]', ' ', str(text).lower().strip())
    text = re.sub('@[A-Za-z0-9_]+', '', text)
    text = re.sub('#','',text)
    text = re.sub('RT[\s]+','',text)
    text = re.sub('https?:\/\/\S+', '', text)
    text = re.sub('\n',' ',text)
    text = emoji.replace_emoji(text, replace='')
    return text

# Apply cleaning function into dataframe
df["Review"] = df["Review"].apply(cleaning)

# Making function for lemmatization process and apply it into dataframe
def lemm(data):
    wordnet = WordNetLemmatizer()
    lemmanized = []
    for i in range(len(data)):
        lemmed = []
        words = word_tokenize(data['Review'].iloc[i])
        for w in words:
            lemmed.append(wordnet.lemmatize(w))
        lemmanized.append(lemmed)

    data['lemmanized'] = lemmanized
    data['Review'] = data['lemmanized'].apply(' '.join)
    data=data.drop("lemmanized",axis=1)
    return data

# define df as df processed bylemm function into
df=lemm(df)

# load the tokenizer
tokenizer = Tokenizer(num_words=5000, split=' ')
tokenizer.fit_on_texts(df['Review'].values)

# padding the sequence
X = tokenizer.texts_to_sequences(df['Review'].values)
X = pad_sequences(X)


In [4]:
# load model
model = tf.keras.models.load_model('model.h5')

# predict the rating
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
[2 2]
